In [ ]:
###
# Importar librerias necesarias para el procesamiento del dataframe
###

'''Pandas para la lectura del archivo'''
import pandas as pd

'''Sns y plt para organizar los graficos visuales'''
import matplotlib.pyplot as plt
import seaborn as sns

#
# Importar el archivo csv del set online llamado Mall_Customers.csv
#
df = pd.read_csv('/content/sample_data/Mall_Customers.csv')

#
# 1. Para este caso de estudio todas las variables pasan por un proceso de
# análisis y se visualizan a través de graficos
#

#
# Visualización básica de los datos para entender las distribuciones
#
sns.pairplot(df.drop('CustomerID', axis=1), hue='Gender')
plt.show()

#
# Visualizar la correlación entre las variables numéricas
#
sns.heatmap(df[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].corr(), annot=True, cmap='coolwarm')
plt.show()


In [ ]:
'''Escaladores de caracteristicas para este caso las variables de estudio'''
from sklearn.preprocessing import StandardScaler, LabelEncoder

#
# 2. Preprocesamiento y Selección de Características
#

#
# Transformación de variables categóricas a numéricas
#
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])

#
# Escalado de características numéricas
#
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']])
df_scaled = pd.DataFrame(df_scaled, columns=['Age', 'Annual Income (k$)', 'Spending Score (1-100)'])

#
# 3. Selección de características
# para este caso de estudio Utilizaremos todas las características numéricas
# en este ejemplo
#
features = df_scaled

#
# 4. Entrenamiento del modelo
#

'''Modelo de ejecución de entrenamiento'''
from sklearn.cluster import KMeans

#
# Encontrar el número óptimo de clusters usando el método del codo
#
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features)
    sse.append(kmeans.inertia_)

plt.plot(range(1, 11), sse, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

#
# Para este caso de estudio tomaremos como número óptimo de
# clusters el valor de 3
#
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(features)

In [ ]:
#
# 5. Evaluación del desempeño
#

'''Metricas de desempeño'''
from sklearn.metrics import silhouette_score, calinski_harabasz_score

labels = kmeans.labels_
print('Silhouette Score:', silhouette_score(features, labels))
print('Calinski-Harabasz Score:', calinski_harabasz_score(features, labels))

In [ ]:
#
# 6. Visualización de resultados
#

#
# Agregar los labels del cluster al dataframe original
#
df['Cluster'] = labels

#
# Visualización de los clusters
#
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', hue='Cluster', data=df, palette='viridis')
plt.title('Cluster Visualization')
plt.show()